In [1]:
%matplotlib inline

~/workspace/fastANI --ql file_listing.txt --rl file_listing.txt --fragLen 300 --minFraction 0.8 -o first_pass_ANI.output

~/workspace/fastANI --ql file_listing.txt --rl file_listing.txt --fragLen 300 --minFraction 0.9 -o fifth_pass_ANI.output

In [2]:
import pandas as pd
from Bio import SeqIO
import glob

In [86]:
input_df = pd.read_csv('./first_pass_ANI.output', header=None, sep='\t')
input_df['temp'] = input_df[0].str.split("/").str[-1]
input_df['query_seq'] = input_df['temp'].str.split('.').str[0]
input_df['temp'] = input_df[1].str.split("/").str[-1]
input_df['ref_seq'] = input_df['temp'].str.split('.').str[0]
input_df = input_df[['query_seq', 'ref_seq', 2, 3, 4]]
print(input_df[input_df[2]==100.].shape)

# input_df = input_df[input_df['query_seq'] != input_df['ref_seq']]
print(input_df.shape)
input_df.head(n=20)

(1045, 5)
(10961, 5)


,query_seq,ref_seq,2,3,4
0,NC_008464,NC_008464,100.0000,200,200
1,minerva,minerva,100.0000,366,366
2,minerva,optimus,96.5329,309,366
3,minerva,wanda,96.4687,317,366
4,minerva,baka,96.2789,309,366
5,island3,island3,100.0000,157,157
6,island3,brujita,99.6888,156,157
7,NC_004587,NC_004587,100.0000,139,139
8,NC_004587,NC_003157,99.9971,139,139
9,NC_024122,NC_024122,100.0000,541,543


In [87]:
records = []
for infile in glob.glob('../../Data/model_data/phage_data_nmicro2017/phage_fasta_files/*.fasta'):
    tempy = SeqIO.read(infile, 'fasta')
    records.append((infile.split('/')[-1].split('.')[0], len(tempy.seq)))
records = sorted(records, key=lambda x: x[1], reverse=True) ###Critical step!
print('First, last record sequence length:', records[0][1], ',', records[-1][1])

First, last record sequence length: 358663 , 5833


In [88]:
thresh = 80
clusters = []
found = []
for record in records[:]: ### Records must be sorted in order! (see above)
    if record[0] in found:
        continue
    ###Find all instances of this record    
    temp_df = input_df[(input_df['query_seq']==record[0]) | (input_df['ref_seq']==record[0])]
    temp_df = temp_df[temp_df[2] > thresh]
    if temp_df.shape[0] != 0:
        all_seqs = list(set(list(temp_df['query_seq'])+list(temp_df['ref_seq'])))
        all_seqs.insert(0, all_seqs.pop(all_seqs.index(record[0])))

        clusters.append(all_seqs)
        found.extend(all_seqs)
        input_df = input_df[input_df['query_seq'].isin(all_seqs)==False]
        input_df = input_df[input_df['ref_seq'].isin(all_seqs)==False]
    else:
        clusters.append([record[0]])
        found.append(record[0])

In [89]:
len(clusters), sum([len(i) for i in clusters])

(511, 1057)

In [152]:
lifestyle_df = pd.read_csv('../../Data/model_data/phage_data_nmicro2017/processed_benchmark_set.csv', index_col=0)
lifestyle_df.shape

(1057, 22)

In [153]:
lifestyle_df['name'] = ''  
lifestyle_df.at[lifestyle_df[lifestyle_df['Database source'] == 'NCBI RefSeq'].index, 'name'] =\
                    lifestyle_df[lifestyle_df['Database source'] == 'NCBI RefSeq']['RefSeq accession number']
lifestyle_df.at[lifestyle_df[lifestyle_df['Database source'] == 'Actinobacteriophage_785'].index, 'name'] =\
                    lifestyle_df[lifestyle_df['Database source'] == 'Actinobacteriophage_785']['Virus identifier used for the analysis'].str.split('_').str[0]

In [154]:
cluster_firsts = [i[0] for i in clusters]
print(len(cluster_firsts))
lifestyle_df = lifestyle_df[lifestyle_df['name'].isin(cluster_firsts)]

647


In [155]:
lifestyle_df = lifestyle_df.drop('name', axis=1)

In [156]:
lifestyle_df.shape

(647, 22)

In [157]:
lifestyle_df.to_csv('../../Data/model_data/phage_data_nmicro2017/processed_benchmark_set_DEREP.csv')

In [65]:
import json

In [90]:
with open('../../Data/model_data/clusters.json', 'w') as outfile:
    json.dump(clusters, outfile)